<a href="https://colab.research.google.com/github/youbkis/YoutubeAudioSeparator/blob/main/Youtube_Audio_Separator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Separate Vocal and Instrument Tracks from Youtube Videos


Based on [Hybrid Demucs](https://colab.research.google.com/drive/1dC9nVxk3V_VPjUADsnFu8EiT-xnU1tGH?usp=sharing) and [ytmp3-dl](https://github.com/poseidon-code/ytmp3-dl)


# Prerequisites

In [1]:
#@title 1. Run to Download Dependencies
%%capture
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs
!pip install -U yt-dlp
!curl -LJO https://raw.githubusercontent.com/poseidon-code/ytmp3-dl/main/ytmp3-dl.py
!chmod +x ytmp3-dl.py
!sudo apt-get install ffmpeg

In [2]:
#@title 2. Run  to Configure Demucs Model
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

In [3]:
#@title 3. Run to Import the download and separation Functions
import io
from pathlib import Path
import select
from shutil import rmtree,copy
import subprocess as sp
import sys
import os
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp, outp):
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")
        return False
    return True


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)

def dl_mp3(link):
    clear_paths()
    try:
      cmd = f"./ytmp3-dl.py -d tmp_in {link}"
      !{cmd}
    except Exception as e:
      print("Error while downloading yt video", e)
      return None
    return "tmp_in/"+os.listdir("tmp_in")[0]

def download_mp3(link):
    mp3 = dl_mp3(link)
    if(mp3 is None):
      return
    files.download(mp3)

def clear_paths():
    out_path = Path('output')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

def separate_from_link(link,keep_original_mp3=False):
    out_path = Path('output')
    in_path = Path('tmp_in')
    clear_paths()
    success = False
    try:
      print(f"Downloading {link}")
      mp3_path = dl_mp3(link)
    except Exception as e:
      print("Halted download:", e)

    print("Download finished.\nSeparating Tracks:")
    success = separate(in_path, out_path)
    if(success):
      print("Separation finished.\nPacking separated tracks into zip file")
      if keep_original_mp3:
        copy(mp3_path, "output")

      !zip -r output.zip output
      files.download('./output.zip')
    else:
      print("Separation failed")





# Converters

In [4]:
#@title Edit the youtube Link and run the cell
LINK = "https://www.youtube.com/watch?v=_XbMXAuEsDU"

In [ ]:
#@title Run to convert the video to mp3
download_mp3(LINK)

In [5]:
#@title Run to separate the audio tracks of the video (may take a few minutes):
keep_original_mp3=True
separate_from_link(LINK,keep_original_mp3)

ytmp3-dl v3.0 ~poseidon-code
| URLs                       : 1
| Using ffmpeg at            : /usr/bin/ffmpeg
| Download Directory         : tmp_in
| Concurrent Download Limit  : 2

[downloading]	 Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song
ytmp3-dl v3.0 ~poseidon-code
| URLs                       : 1
| Using ffmpeg at            : /usr/bin/ffmpeg
| Download Directory         : tmp_in
| Concurrent Download Limit  : 2

[finished]	 Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song
Download finished.
Separating Tracks:
Going to separate the files:
tmp_in/Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song.mp3
With command:  python3 -m demucs.separate -o output -n htdemucs --mp3 --mp3-bitrate=320


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 106MB/s]


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/output/htdemucs
Separating track tmp_in/Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song.mp3


100%|██████████████████████████████████████████████| 269.09999999999997/269.09999999999997 [00:17<00:00, 15.79seconds/s]


Separation finished.
Packing separated tracks into zip file
  adding: output/ (stored 0%)
  adding: output/Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song.mp3 (deflated 1%)
  adding: output/htdemucs/ (stored 0%)
  adding: output/htdemucs/Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song/ (stored 0%)
  adding: output/htdemucs/Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song/bass.mp3 (deflated 3%)
  adding: output/htdemucs/Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song/other.mp3 (deflated 3%)
  adding: output/htdemucs/Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song/vocals.mp3 (deflated 2%)
  adding: output/htdemucs/Гаухартас 'Қазағым ай' Kazakh Turkic Turanian Song/drums.mp3 (deflated 2%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>